<a href="https://colab.research.google.com/github/paulademelo/Fatec-DSM/blob/main/ProjetoFinal_Minera%C3%A7%C3%A3o_Turismo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto Final Mineração de Dados
### Base de dados: **Pontos de cultura rede cultura viva**
https://dados.turismo.gov.br/dataset/pontos-de-cultura-rede-cultura-viva

In [ ]:
# Preparando o ambiente
import pandas as pd
import numpy as np

In [58]:
# Importando a base de dados
dados = pd.read_excel('/content/dados-turismo-roda-viva.xlsx')

# Trabalhando com a Limpeza


In [ ]:
# Visualizar as primeiras linhas dos dados
dados.head(50)

In [ ]:
# Apagando as primeiras linhas desnecessárias
dados_limpos = dados.drop([0, 1])

In [ ]:
len(dados_limpos)

4323

In [ ]:
# Verificando se o índice são os títulos
dados_limpos.head(0)

In [ ]:
# Removendo a linha 1
dados.drop(1, axis = 0, inplace = True)

In [ ]:
# Imprimindo as primeiras 50 linhas da coluna
dados.head(50)

In [ ]:
# Verificar informações sobre os dados
dados.info()

In [ ]:
# Verificar estatísticas descritivas
dados.describe()

In [61]:
# Verificar quantos valores ausentes existem em cada coluna
dados.isnull().sum()

Id                                        0
Nome                                      0
Localização                               0
Descrição Longa                         592
Data de Criação                           0
ID usuário                                0
Data de Atualização                       4
Tipo de agente da Rede Cultura Viva       0
Estado                                    0
Município                                 0
Bairro                                   50
Número                                   19
Endereço                               1051
Ações Estruturantes                    2898
Públicos que participam das ações      3024
Área de experiência e temas            2957
dtype: int64

In [ ]:
# Verificar e remover duplicatas
dados = dados.drop_duplicates()

In [ ]:
# Verificar novamente quantos valores ausentes existem em cada coluna
dados.isnull().sum()

In [ ]:
dados2 = dados.dropna(how='all', axis = 0)

In [ ]:
dados2.describe()

In [ ]:
# mostrando o tamanho da tabela, no caso da nossa base de dados:
# 4324 linhas e 17 colunas
dados2.shape

(4324, 17)

In [ ]:
# retorna uma Series que exibe o tipo de dado de cada coluna
dados2.dtypes

In [ ]:
# Explorando as colunas
dados2.head()

In [ ]:
dados2

In [70]:
# Substituir os valores NaN na coluna 'Localização' pelo próprio nome
dados['Descrição Longa'].fillna(dados['Nome'], inplace=True)

# Substituir os valores nulos na coluna "Número" por "s/n"
dados['Número'] = dados['Número'].fillna('s/n')

# Substituir os valores nulos na coluna "Endereço" pelo nome do município
dados['Endereço'] = dados['Endereço'].fillna(dados['Município'])

# Substituir os valores nulos na coluna "Bairro" pelo nome do município
dados['Bairro'] = dados['Bairro'].fillna(dados['Município'])

# Substituir os valores nulos na coluna "Data de atualização" pela data de criação
dados['Data de Atualização'] = dados['Data de Atualização'].fillna(dados['Data de Criação'])

# Adicionando o valor "cultura" que é padrão aos pontos na coluna 'Ações Estruturantes' para tratar os nulos
dados['Ações Estruturantes'] = dados['Ações Estruturantes'].fillna('cultura')

# Adicionando o valor "público em geral" que é padrão aos pontos na coluna 'Públicos que participam das ações' para tratar os nulos
dados['Públicos que participam das ações'] = dados['Públicos que participam das ações'].fillna('público em geral')

# Adicionando o valor "cultural" que é padrão aos pontos na coluna 'Área de experiência e temas' para tratar os nulos
dados['Área de experiência e temas'] = dados['Área de experiência e temas'].fillna('cultural')

In [71]:
# Validando novamente se há algum dado nulo
dados.isnull().sum()

Id                                     0
Nome                                   0
Localização                            0
Descrição Longa                        0
Data de Criação                        0
ID usuário                             0
Data de Atualização                    0
Tipo de agente da Rede Cultura Viva    0
Estado                                 0
Município                              0
Bairro                                 0
Número                                 0
Endereço                               0
Ações Estruturantes                    0
Públicos que participam das ações      0
Área de experiência e temas            0
dtype: int64

In [72]:
# Exportando a tabela pós-limpeza
dados.to_excel('pontos-cultura-pos-processamento.xlsx', index=False)

## Trabalhando com gráficos e representações visuais

In [73]:
import folium

# Criei um mapa central de acordo com a primeira linha
mapa = folium.Map(location=[-19.9007755, -44.0836458], zoom_start=2)

# Iterar sobre os dados da coluna "Localização" e adicionar marcadores para cada ponto
for localizacao in dados['Localização']:
    if isinstance(localizacao, str):  # Verificar se o valor é uma string
        lat, lon = map(float, localizacao.split(','))  # Dividir a localização em latitude e longitude
        folium.Marker([lat, lon]).add_to(mapa)  # Adicionar marcador com base na latitude e longitude

mapa


In [ ]:
# Criando um gráfico para mostrar a quantidade de pontos por estado
from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['Estado']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'Estado'}, axis=1)
              .sort_values('Estado', ascending=True))
  xs = counted['Estado']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(20, 10.2), layout='constrained')
df_sorted = dados_limpos.sort_values('Estado', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Estados')
_ = plt.ylabel('count()')